<a href="https://colab.research.google.com/github/ge96lip/Quantum-Computing/blob/main/QC_Shor's_Factorization_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [349]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit_aer

In [350]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Qubit
from qiskit.circuit.library import XGate, CSwapGate, QFT
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

import matplotlib.pyplot as plt
import math
import time

# Shor's Factorization Algorithm

In [351]:
def ShorLogic(N, repeat_period_candidates, coprime):
    print('Repeat period candidates: '+str(repeat_period_candidates)+'\n')
    factor_candidates = []
    for i in range(len(repeat_period_candidates)):
        repeat_period = repeat_period_candidates[i];
        # Given the repeat period, find the actual factors
        ar2 = pow(coprime, repeat_period / 2.0)
        factor1 = int(gcd(N, ar2 - 1))
        factor2 = int(gcd(N, ar2 + 1))
        factor_candidates.append([factor1, factor2])
    return factor_candidates

In [352]:
def gcd(a, b):
    # return the greatest common divisor of a,b
    while b:
        m = a % b
        a = b
        b = m
    return a

In [353]:
def check_result(N, factor_candidates):
    for i in range(len(factor_candidates)):
        factors = factor_candidates[i]
        if factors[0] * factors[1] == N:
            if factors[0] != 1 and factors[1] != 1:
                # Success!
                return factors
    # Failure
    return None

In [354]:
def Shor(N, coprime, classical=False, vis=False, opt_for_vis=False):
    # Quantum part
    if classical:
        # Classical alternative for comparision
        repeat_period = ShorNoQPU(N, coprime)
    else:
        # Quantum part
        repeat_period = ShorQPU(N, coprime, vis, opt_for_vis)

    # Classical part
    factors = ShorLogic(N, repeat_period, coprime)

    return check_result(N, factors)

In [355]:
def ShorNoQPU(N, coprime):
    # Classical replacement for the quantum part of Shor
    repeat_period_candidates = []
    work = 1
    precision_bits = math.ceil(math.log2(N))
    max_loops = pow(2, precision_bits)
    for iter in range(max_loops):
        work = (work * coprime) % N
        if work == 1: # found a repeat period
            repeat_period_candidates.append(iter + 1)
    return repeat_period_candidates

In [356]:
def ShorQPU(N, coprime, vis=False, opt_for_vis=False):
    # Quantum part of Shor's algorithm
    # For this implementation, the coprime must be 2.
    coprime = 2;
    return ShorQPU_WithModulo(N, coprime, vis, opt_for_vis)

    '''
    # For some numbers (like 15 and 21) the "mod" in a^xmod(N)
    # is not needed, because a^x wraps neatly around. This makes the
    # code simpler, and much easier to follow.
    if N == 15 or N == 21:
        return ShorQPU_WithoutModulo(N, coprime, opt_for_vis)
    else:
        return ShorQPU_WithModulo(N, coprime, vis, opt_for_vis)
    '''

In [357]:
def RollLeft(qc, work, num_shifts, control):
    # Ensure we're not out of range for work qubits
    for j in range(work.size - num_shifts-1,-1,-1):
        # Apply CSWAP between work[j] <-> work[j + num_shifts]
        qc.append(CSwapGate(), [control, work[j], work[j + num_shifts]])
    return qc

In [358]:
# In case our QPU read returns a "signed" negative value,
# convert it to unsigned.
def read_unsigned(qc, qreg, creg):
    qc.measure(qreg, creg)

    # Simulate the circuit using the Aer simulator
    simulator = AerSimulator()
    job = simulator.run(transpile(qc, simulator), shots=1)
    result = job.result()
    counts = result.get_counts(qc)

    # Visualize the results
    #print("Measurement results: " + str(counts) + "\n")
    #fig = plot_histogram(counts)
    #fig.show()

    # Extract value of most likely measurement outcome
    value = int(max(counts, key=counts.get),2)

    return value & ((1 << qreg.size) - 1)

In [359]:
def estimate_num_spikes(spike, spike_range):
    if spike < spike_range / 2:
        spike = spike_range - spike
    best_error = 1.0
    e0 = 0
    e1 = 0
    e2 = 0
    actual = spike / spike_range
    candidates = []
    for denominator in range(1,spike):
        numerator = round(denominator * actual)
        estimated = numerator / denominator
        error = abs(estimated - actual)
        e0 = e1
        e1 = e2
        e2 = error
        # Look for a local minimum which beats our current best error
        if e1 <= best_error and e1 < e0 and e1 < e2:
            repeat_period = denominator - 1
            candidates.append(repeat_period)
            best_error = e1
    return candidates

In [360]:
# This is the short/simple version of ShorQPU() where we can perform a^x and
# don't need to be concerned with performing a quantum int modulus.

def ShorQPU_WithoutModulo(N, coprime, vis=False, opt_for_vis=False):

    # Define number of required qubits
    N_bits = math.ceil(math.log2(N))
    work_qubits = N_bits + 1
    precision_qubits = N_bits

    # Calculate size of working register
    #N_bits = 1
    #while (1 << N_bits) < N:
    #    N_bits +=1
    #if N != 15: # For this implementation, numbers other than 15 need an extra bit
    #    N_bits +=1

    # Set up the QPU and the working registers
    work = QuantumRegister(work_qubits, name="work")
    precision = QuantumRegister(precision_qubits, name="precision")
    classical = ClassicalRegister(precision_qubits, name="precision_measure")
    qc = QuantumCircuit(work, precision, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.h(precision)

    if opt_for_vis: qc.barrier()

    # Perform 2^x for all possible values of x in superposition
    for iter in range(precision_qubits):
        num_shifts = 1 << iter
        if num_shifts < work_qubits:
            qc = RollLeft(qc, work, num_shifts, precision[iter])
            if opt_for_vis: qc.barrier()

    # Quantum Fourier Transform on precision register
    qc.append(QFT(precision_qubits), precision)

    if opt_for_vis: qc.barrier()

    if vis: print(qc)

    read_result = read_unsigned(qc, precision, classical)
    print('QPU read result: '+str(read_result)+'\n')
    repeat_period_candidates = estimate_num_spikes(read_result, 1 << precision_qubits)

    print('Circuit depth: ', qc.depth())
    print('Number of qubits: ', qc.num_qubits)

    return repeat_period_candidates

In [361]:
def add_int(qc, qreg, N, conditions):
    reverse_to_subtract = False
    if N == 0:
        return
    elif N < 0:
        N = -N
        reverse_to_subtract = True
    ops = []
    add_val = int(N)
    condition_mask = (1 << len(qreg)) - 1

    add_val_mask = 1
    while add_val_mask <= add_val:
        cmask = condition_mask & ~(add_val_mask - 1)
        if add_val_mask & add_val:
            add_shift_mask = 1 << (len(qreg) - 1)
            while add_shift_mask >= add_val_mask:
                cmask &= ~add_shift_mask
                ops.append((add_shift_mask, cmask))
                add_shift_mask >>= 1
        condition_mask &= ~add_val_mask
        add_val_mask <<= 1
    if reverse_to_subtract:
        ops.reverse()
    for inst in ops:
        op_qubits = []
        for i in range(len(qreg)):
            if inst[1] & (1 << i):
                op_qubits.append(qreg[i])
        for i in range(len(qreg)):
            if inst[0] & (1 << i):
                op_qubits.append(qreg[i])
        qc = multi_cx(qc, op_qubits, conditions)

    return qc


def multi_cx(qc, qubits, conditions=None, do_cz=False):
    ## This will perform a CCCCCX with as many conditions as we want
    ## The last qubit in the list is the target.

    target = qubits[-1]
    conds = qubits[:-1]
    ops = []

    controll_qubits = []

    for i in range(len(conds)):
        controll_qubits.append(conds[i])
    if conditions != None:
        for i in range(len(conditions)):
            controll_qubits.append(conditions[i])

    if len(controll_qubits) == 0:
        qc.x(target)
    else:
        qc.mcx(controll_qubits, target)

    return qc

In [362]:
# This is the complicated version of ShorQPU() where we DO
# need to be concerned with performing a quantum int modulus.
# That's a complicated operation, and it also requires us to
# do the shifts one at a time.
def ShorQPU_WithModulo(N, coprime, vis=False, opt_for_vis=False):
    scratch = None
    max_value = 1
    mod_engaged = False

    # Define number of required qubits
    N_bits = math.ceil(math.log2(N))
    work_qubits = N_bits + 1
    precision_qubits = N_bits
    scratch_qubits = 1

    #N_bits = 1
    #scratch_bits = 0
    #while (1 << N_bits) < N:
    #    N_bits +=1
    #print('N_bits: '+str(N_bits)+' log: '+str(math.ceil(math.log2(N))))
    #if N != 15: # For this implementation, numbers other than 15 need an extra bit
    #    N_bits +=1


    # Set up the QPU and the working registers
    work = QuantumRegister(work_qubits, name="work")
    precision = QuantumRegister(precision_qubits, name="precision")
    scratch = QuantumRegister(scratch_qubits, name="scratch")
    classical = ClassicalRegister(precision_qubits, name="precision_measure")
    qc = QuantumCircuit(work, precision, scratch, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.initialize(0, scratch)
    qc.h(precision)

    if opt_for_vis: qc.barrier()

    N_sign_bit = work[work_qubits - 1]

    for iter in range(precision_qubits):
        condition_qubit = precision[iter]  # Get the condition qubit

        shifts = 1 << iter
        for shift in range(shifts):
            qc = RollLeft(qc, work, 1, precision[iter]) # multiply by the coprime 2
            if opt_for_vis: qc.barrier()
            max_value <<= 1

            if max_value >= N:
                mod_engaged = True

            if mod_engaged:
                qc = add_int(qc, work, -N, [condition_qubit]) # Substract N from working register, causing this to go negative if work < N
                if opt_for_vis: qc.barrier()
                qc.ccx(N_sign_bit, condition_qubit, scratch) # Skim off the sign bit
                if opt_for_vis: qc.barrier()
                qc = add_int(qc, work, N, [scratch[0], condition_qubit]) # If we went negative, undo the subtraction by adding N
                if opt_for_vis: qc.barrier()
                qc.x(work[0])
                qc.ccx(work[0], condition_qubit, scratch) # If it's odd, then we wrapped, so clear the wrap bit
                qc.x(work[0])
                if opt_for_vis: qc.barrier()

    # Quantum Fourier Transform on precision register
    qc.append(QFT(precision_qubits), precision)

    if vis: print(qc)

    read_result = read_unsigned(qc, precision, classical)
    print('QPU read result: ', read_result)
    repeat_period_candidates = estimate_num_spikes(read_result, 1 << precision_qubits)

    print('Circuit depth: ', qc.depth())
    print('Number of qubits: ', qc.num_qubits)

    return repeat_period_candidates

# Execution

**Here are some values of N to try:**

15, 21, 33, 35, 39, 51, 55, 69, 77, 85, 87, 91, 93, 95, 111, 115, 117,
119, 123, 133, 155, 187, 203, 221, 247, 259, 287, 341, 451

**Larger numbers require more bits of precision.**

N = 15    precision_bits >= 4

N = 21    precision_bits >= 5

N = 35    precision_bits >= 6

N = 123   precision_bits >= 7

N = 341   precision_bits >= 8  time: about 6 seconds

N = 451   precision_bits >= 9  time: about 23 seconds

In [364]:
def shor_sample(classical=False, vis=False, opt_for_vis=False):
    N = 123             # The number we're factoring
    coprime = 2        # Needs to be 2 in this implementation

    max_iteration = 10
    success = 0
    calculation_time = []

    for i in range(max_iteration):
        start_time = time.time()
        result = Shor(N, coprime, classical, vis, opt_for_vis)
        end_time = time.time()
        if result != None:
            success += 1
            calculation_time.append(end_time-start_time)
            print('Iteration '+str(i+1)+': Successful'+'. Calculation time: '+str("{:.2f}".format(end_time-start_time))+'s'+'. Success: '+str(success)+'/'+str(max_iteration))

            if success == 1: # Store solution
                solution = str(N)+'='+str(result[0])+'*'+str(result[1])
        else:
            print('Iteration '+str(i+1)+': Not Successful'+'. Success: '+str(success)+'/'+str(max_iteration))
        print('-----------------------------------------------------------------------------------\n')

    if success == 0:
        print('Failure: No non-trivial factors were found.\n')
    else:
        success_rate = success/max_iteration
        average_time = sum(calculation_time)/len(calculation_time)
        print('Solution: '+solution+'\nAccuracy: '+str(success_rate*100)+'%'+'\nAverage calculation time: '+str("{:.2f}".format(average_time))+'s')

# 'vis=True' shows quantum circuit
# 'vis=False' does not show quantum circuit
# 'opt_for_vis=True' adds barriers to the circuit to improve readability
# 'opt_for_vis=False' removes barriers in the circuit to improve performance
shor_sample(classical=False, vis=False, opt_for_vis=False)

QPU read result:  19
Circuit depth:  8139
Number of qubits:  16
Repeat period candidates: [7, 13, 20, 27, 54, 74, 101]

Iteration 1: Successful. Calculation time: 6.44s. Success: 1/10
-----------------------------------------------------------------------------------

QPU read result:  7
Circuit depth:  8139
Number of qubits:  16
Repeat period candidates: [18, 37, 55, 73]

Iteration 2: Not Successful. Success: 1/10
-----------------------------------------------------------------------------------

QPU read result:  45
Circuit depth:  8139
Number of qubits:  16
Repeat period candidates: [3, 6, 9, 11, 14, 17, 34, 37, 74]

Iteration 3: Not Successful. Success: 1/10
-----------------------------------------------------------------------------------

QPU read result:  26
Circuit depth:  8139
Number of qubits:  16
Repeat period candidates: [5, 10, 15, 20, 25, 30, 34, 39, 44, 49, 54, 59, 64]

Iteration 4: Successful. Calculation time: 5.76s. Success: 2/10
------------------------------------